# README
This document does not correctly solve the problem. No constraint equation found for the L-shape of hooks.

Instead, see `main.py`, where all hook configurations are generated, and `NumberPlacementSolver.py` is used to place the values within the hooks.

#### Load CP-SAT and test values

In [1]:
from ortools.sat.python import cp_model
model = cp_model.CpModel()
solver = cp_model.CpSolver()

from tests import ex_vals, ex_vals_list, ex_hook, ex_grid
grid = ex_grid
vals_list = ex_vals_list
N, _ = grid.shape

#### Problem setup
**Grid sum**
* The matrix is filled with one 1's, two 2's, ..., and $N$-many $N$'s. 
* The sum of all values is $\sum_{i=1}^{N} i^2 = \frac{N (N+1) (2N+1)}{6}$.

In [2]:
from itertools import product

ngrids = grid.max()
gridsum_target = N*(N+1)*(2*N+1)//(6*ngrids)
num_adjs = N * (N+1) - 2

Nrange = [i+1 for i in range(N)] # (1 to N)
Vrange = [i for i in range(N+1)] # value (0 to N)
Grange = [i+1 for i in range(ngrids)] # grid index
Arange = [i for i in range(5)] # number of adjacents (0 to 4)

RC   = list(product(*[Nrange]*2))
RCHV = list(product(*[Nrange]*3, Vrange))
RCHA = list(product(*[Nrange]*3, Arange))

#### Setup variables:
* $x_{r,c,h,v} = 1$ $\Leftrightarrow$ cell $(r,c)$ belongs to hook $h$ and contains value $v$.
    * Note that $v=0$ is used to indicate that no value is in a cell.
* $y_{r,c,h,a} = 1$ $\Leftrightarrow$ cell $(r,c)$ belongs to hook $h$ and has $\leq a$ adjacent non-zero values.
    * The idea of the $\leq a$ is so that adjacencies of cells with 0's can potentially be ignored by setting them to 0.
    * For the matrix to contain a single connected component while satisfying the sparsity constraint (i.e. at least one zero in every $2 \times 2$ sub-matrix), the total number of adjacencies is $n_{adj} = N (N+1) - 2$. 
        * The proof is straight-forward using recursion. Consider a component of size $k$ with $n_{adj}(k)$ adjacencies. 
        * Adding the $(k+1)$-th value adds exactly 2 adjacencies: one for to the new value added and one for the component of size $k$ --- recall the $(k+1)$-th value cannot touch two existing components, since that will violate the sparsity constraint. 
        * This gives $n_{adj} (k+1) = n_{adj} (k) + 2$. 
        * With $n_{adj} (1) = 0$, it follows that $n_{adj} (k) = 2 (k-1) $.
        * With one 1, two 2, ..., $N$-many $N$'s, there are a total of $\sum_{i=1}^{N} i = \frac{1}{2} N(N+1)$ values, so the total adjacencies is $N (N+1) - 2$
* $m_{h,v} = 1$ $\Leftrightarrow$ cells in hook $h$ can only contain $v$ (or no value at all).
    * Value of $m_{h,0}$ is irrelevant

In [3]:
x = {}
for r,c,h,v in RCHV:
    x[r,c,h,v] = model.NewBoolVar(f'x[{r},{c},{h},{v}]')

y = {}
for r,c,h,a in RCHA:
    y[r,c,h,a] = model.NewBoolVar(f'y[{r},{c},{h},{a}]')

m = {}
for h,v in product(Nrange, Vrange):
    m[h,v] = model.NewBoolVar(f'm[{h},{v}]')

## Max number of adjacencies
for r,c in RC:
    counted_adjs = sum(a*y[r,c,h,a] for h in Nrange for a in Arange)
    max_adjs = sum(
        (x[r-1,c  ,h,v] if r-1>=1 else 0) +
        (x[r+1,c  ,h,v] if r+1<=N else 0) +
        (x[r  ,c-1,h,v] if c-1>=1 else 0) +
        (x[r  ,c+1,h,v] if c+1<=N else 0)
        for h in Nrange for v in Vrange if v > 0)
    model.Add(counted_adjs <= max_adjs)

# (h) should correspond between x and y for each (r,j)
for r,c in RC:
    for h in Nrange:
        model.Add(sum(y[r,c,h,a] for a in Arange) == sum(x[r,c,h,v] for v in Vrange if v > 0))
        # pass

## CONNECTEDNESS constraint (2(N-1) total adjacencies after double counting, assuming SPARSITY constraint holds)
adjs = (a*y[r,c,h,a] for r,c in RC for h in Nrange for a in Arange)
model.Add(sum(adjs) == num_adjs)

# only points with non-zero values should have neighbors (minimise number of activated y)
objective = sum([y[r,c,h,a] for r,c,h,a in RCHA])
model.Minimize(objective)

#### Uniqueness constraints
* Each cell $(r,c)$ should only be associated with one hook $h$ and one value $v$.
* Each cell $(r,c)$ should only be associated with at most one $a$ (max number of adjacencies).
* Each hook $h$ maps exactly to a non-zero $v$, and vice-versa.

In [4]:
## UNIQUENESS constraints
# each (r,c) has one unique (h,v)
for r,c in RC:
    model.AddExactlyOne(x[r,c,h,v] for h in Nrange for v in Vrange)
# each (r,c) cannot have multiple max adjacencies
for r,c in RC:
    model.AddAtMostOne(y[r,c,h,a] for h in Nrange for a in Arange) 
# each (h) maps to 1 positive (v)
for v in Vrange:
    if v > 0:
        model.AddExactlyOne(m[h,v] for h in Nrange)
for h in Nrange:
        model.AddExactlyOne(m[h,v] for v in Vrange if v > 0)

#### Sparsity constraint
For each $2 \times 2$ sub-matrix, check that the total number non-zero values is no more than 3.

In [5]:
## SPARSITY constraint (one zero in every 2x2 submatrix)
for r,c in RC:
    if r < N and c < N:
        mat22 = (x[r  ,c,h,v] + x[r  ,c+1,h,v] +
                 x[r+1,c,h,v] + x[r+1,c+1,h,v]
                 for h in Nrange for v in Vrange if v > 0)
        # model.Add(sum(mat22) <= 3)

#### Constrain hooks
**Shape**
* Each hook of size $h$ has exactly $4h-4$ adjacencies (each of $2h-3$ interior cells have $2$, and each of the $2$ end cells have $1$).
* Each cell in a hook can have at most 2 adjacencies (i.e. no T's or crossroads).
* Each (h) has exactly one turn.
    * A turn is defined if cell $s=(r,c)$ is non-zero and either $e=(r-1, c+1)$ or $e'=(r+1, c+1)$ is non-zero. 

        ```
        s     e   sx   xe
        xe'  sx    e'  s
        ```
**Values**
* If hook $h$ is associated with value $v>0$, then it contains exactly $v$-many $v$'s.

In [6]:
## HOOK constraints
for h in Nrange:
    # Shape
    # model.Add(sum(a*y[r,c,h,a] for r,c in RC for a in Arange) == 4*h-4)
    for r,c in RC:
        model.Add(sum(a*y[r,c,h,a] for a in Arange) <= 2)
    # (2h-1)E(ij) - E(i)E(j) = (h-1)^2 h^2 / 4
    model.Add((2*h-1)*sum(r*c*x[r,c,h,a] for r,c in RC) - sum(r*x[r,c,h,a] for r,c in RC) * sum(c*x[r,c,h,a] for r,c in RC) == (h-1)**2 * h**2 // 4)

    # Value
    for v in Vrange:
        if v > 0:
            model.Add(sum(x[r,c,h,v] for r,c in RC) == v*m[h,v])

#### Instance constraints
* Array $[i,j]$ contains value $v'$:
    * $\sum_h x_{i+1, j+1, h, v'} = 1$
* Sum of values in each sub-grid $g$ sums to the correct value:
    * $\sum_{h,v} \sum_{(r', c') \in g} v x_{r',c',h,v} = \frac{N(N-1)(2N+1)}{6 n_{grid}}$ 

In [7]:
## INSTANCE constraints
# prescribed values
for (i,j), val in vals_list:
    model.AddExactlyOne(x[i+1,j+1,h,val] for h in Nrange)
# correct grid sum
for g in Grange:
    subgrid_vals = (v*x[r,c,h,v] for r,c,h,v in RCHV if grid[r-1,c-1] == g)
    model.Add(sum(subgrid_vals) == gridsum_target)

#### Solve

In [8]:
from main_cpsat import assignSol, checkVV, checkAdj
status = solver.Solve(model)
if status == cp_model.INFEASIBLE:
    print('No solution found')
elif status == cp_model.FEASIBLE or status == cp_model.OPTIMAL:
    X,Y,A,M,H,V = assignSol(solver, N, x, y, m)
    if not checkVV(V):
        print('VV unsatisifed')
    if not checkAdj(V):
        print('Total adjacencies unsatisifed')

    print('-------hook------')
    for row in H:
        print(row)
    print('-------val------')
    for row in V:
        print(row)
else:
    print(f'Unknown status: {status}')

VV unsatisifed
Total adjacencies unsatisifed
-------hook------
[1 2 1 5 5]
[1 2 3 1 1]
[1 1 1 1 1]
[3 5 2 2 1]
[1 1 4 1 1]
-------val------
[0 4 5 3 3]
[0 4 2 5 5]
[0 0 0 0 0]
[2 3 4 4 0]
[5 0 1 5 0]


In [9]:
checkAdj(ex_vals)

True